
# 📘 Notebook 2 — Feature Engineering for Hotel Demand Forecasting (2015–2025)

This notebook builds upon the cleaned dataset to construct **temporal**, **macroeconomic**, and **lag-based features** for forecasting hotel demand across EU countries.  
It prepares a modeling-ready dataset that serves as the foundation for econometric and machine-learning analyses in **Notebook 3**.

---

### **Structure Overview**

0. Environment Setup  
1. Load Dataset  
2. Feature Engineering: Lag Variables  
3. Feature Engineering: Month-on-Month (MoM) Changes  
4. Temporal Features  
5. Data Quality Check  
6. Export Feature-Engineered Dataset  

---

**Input:**  
📁 `../data/processed/hotel_clean.csv`

**Output:**  
📁 `../data/processed/hotel_features.csv`


In [1]:
# %% ===============================================================
# STEP 0 — ENVIRONMENT SETUP
# ===============================================================
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("viridis")

BASE_DIR = Path("..")
DATA_PROCESSED = BASE_DIR / "data" / "processed"

for path in [DATA_PROCESSED]:
    path.mkdir(parents=True, exist_ok=True)

print("✅ Environment setup complete.")

✅ Environment setup complete.


In [ ]:
# %% ===============================================================
# STEP 1 — LOAD DATASET
# Purpose: Load the cleaned hotel dataset produced in Notebook 1.
# ===============================================================

DATA_PATH = DATA_PROCESSED / "hotel_clean.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["month"])

print(f"✅ Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"✅ Range: {df['month'].min().strftime('%Y-%m')} → {df['month'].max().strftime('%Y-%m')}")
print(f"✅ Countries: {df['region'].nunique()}")

✅ Dataset loaded: 3,328 rows × 10 columns
✅ Range: 2015-01 → 2025-08
✅ Countries: 26


In [ ]:
# %% ===============================================================
# STEP 2 — FEATURE ENGINEERING: LAG VARIABLES
# Purpose: Create lag features for selected exogenous variables (L = 1, 2, 3).
# ===============================================================

lag_features = [
    "log_gdp", 
    "unemployment_rate", 
    "turnover_index", 
    "weighted_stringency_index", 
    "eurusd", 
    "eurgbp"
]

for col in lag_features:
    for L in [1, 2, 3]:
        df[f"{col}_lag{L}"] = df.groupby("region")[col].shift(L)

print(f"✅ Added {len(lag_features)*3} lag features.")

✅ Added 18 lag features.


In [ ]:
# %% ===============================================================
# STEP 3 — FEATURE ENGINEERING: MONTH-ON-MONTH (MoM) CHANGES
# Purpose: Capture monthly growth dynamics.
# ===============================================================

mom_vars = ["log_gdp", "turnover_index"]

for col in mom_vars:
    df[f"{col}_mom"] = (
        df.groupby("region")[col].pct_change() * 100
    )

df.replace([np.inf, -np.inf], np.nan, inplace=True)

print(f"✅ Added {len(mom_vars)} MoM change features.")

✅ Added 2 MoM change features.


In [ ]:
# %% ===============================================================
# STEP 4 — TEMPORAL FEATURES
# Purpose: Extract time-based components to capture seasonality.
# ===============================================================

df["m"] = df["month"].dt.month
df["quarter"] = df["month"].dt.quarter

print("✅ Added time-based features (month, quarter).")

✅ Added time-based features (month, quarter).


In [ ]:
# %% ===============================================================
# STEP 5 — DATA QUALITY CHECK
# Purpose: Summarize missing values and basic descriptive stats..
# ===============================================================

# --- Missing values summary ---
missing_tbl = (
    df.isna()
    .sum()
    .reset_index()
    .rename(columns={"index": "column", 0: "missing_count"})
)
missing_tbl["missing_pct"] = (missing_tbl["missing_count"] / len(df) * 100).round(2)

print("✅ Missing Values Summary:")
display(missing_tbl.sort_values("missing_count", ascending=False).head(10))

# --- Quick numeric overview ---
print("\n✅ Numeric Feature Summary:")
display(df.describe().T.round(2))

✅ Missing Values Summary:


,column,missing_count,missing_pct
18,turnover_index_lag3,78,2.34
27,eurgbp_lag3,78,2.34
24,eurusd_lag3,78,2.34
21,weighted_stringency_index_lag3,78,2.34
12,log_gdp_lag3,78,2.34
15,unemployment_rate_lag3,78,2.34
20,weighted_stringency_index_lag2,52,1.56
14,unemployment_rate_lag2,52,1.56
17,turnover_index_lag2,52,1.56
26,eurgbp_lag2,52,1.56



✅ Numeric Feature Summary:


,count,mean,min,25%,50%,75%,max,std
month,3328,2020-05-15 15:22:30,2015-01-31 00:00:00,2017-09-22 12:00:00,2020-05-15 12:00:00,2023-01-07 18:00:00,2025-08-31 00:00:00,NaN
year,3328.0,2019.84375,2015.0,2017.0,2020.0,2022.25,2025.0,3.083779
log_nights_spent,3328.0,13.448212,5.147494,12.100265,13.632071,14.594523,17.22325,1.812729
log_gdp,3328.0,10.389093,7.427144,9.032616,10.493316,11.423246,13.415018,1.561592
unemployment_rate,3328.0,6.672356,2.0,4.6,6.0,7.7,25.1,3.238143
turnover_index,3328.0,119.753696,6.3,78.7,114.5,155.525,419.6,56.704813
weighted_stringency_index,3328.0,49.18058,36.617093,44.671611,48.849586,52.96561,65.876549,6.46075
eurusd,3328.0,1.118962,0.982956,1.084225,1.115387,1.157142,1.240957,0.051836
eurgbp,3328.0,0.849934,0.701,0.842647,0.859745,0.88004,0.9201,0.049174
log_gdp_lag1,3302.0,10.388227,7.427144,9.032162,10.49297,11.421645,13.415018,1.561746


In [ ]:
# %% ===============================================================
# STEP 6 — EXPORT FEATURE-ENGINEERED DATASET
# Purpose: Save all relevant columns as a modeling-ready dataset.
# ===============================================================

# --- Identify feature columns ---
lag_cols = [c for c in df.columns if "_lag" in c or "_mom" in c]

# --- Define export column order ---
export_cols = [
    "region", "month", "year", "log_nights_spent"
] + lag_cols + [
    "log_gdp", "unemployment_rate", "turnover_index",
    "weighted_stringency_index",
    "eurusd", "eurgbp"
]

# --- Create a clean feature DataFrame ---
df_features = df[export_cols].copy()

# --- Save feature-engineered dataset ---
FEATURE_PATH = DATA_PROCESSED / "hotel_features.csv"
df_features.to_csv(FEATURE_PATH, index=False)

# --- Confirm output ---
print(f"💾 Feature-engineered dataset saved → {FEATURE_PATH.resolve()}")
print(f"✅ Shape: {df_features.shape[0]} rows × {df_features.shape[1]} columns")

💾 Feature-engineered dataset saved → /Users/golibsanaev/Library/CloudStorage/Dropbox/GitHub_gsanaev/forecasting-explaining-hotel-demand-in-eu/data/processed/hotel_features.csv
✅ Shape: 3328 rows × 30 columns


## ✅ Summary

Notebook 2 completed successfully.  
The feature-engineered dataset is ready for model training and evaluation in **Notebook 3 — Model Estimation and Comparison (Econometric + ML Models)**.

---